In [1]:
import json
import pandas as pd
path = 'results/bula_all.jsonl'

mini_paciente = pd.read_csv('datasets/mini_paciente.csv')

list_json = []

with open(path, 'r') as file:
    for line in file:
        line_json = json.loads(line)
        line_json['full_topic'] = mini_paciente.loc[mini_paciente['id'] == line_json['id'], 'full_topic'].values
        list_json.append(line_json)

In [2]:
from bula_process.models import Mistral, Rag, Llama3
model = Mistral()
rag = Rag()

/home/navarro/miniconda3/envs/python310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/navarro/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.45it/s]
/home/navarro/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/navarro/.local/lib/python3.10/site-packages/huggingface_hub

In [11]:
def make_question(id):
    data = list_json[id]
    context = rag.retrieve(data['query'])
    question_prompt = f"Given this context: {context} \n Answer the question:{data['query']}"
    answer_model = model.inference(question_prompt)
    print(f"Pergunta: {data['query']}")
    print('\n')
    print("Alternativas:")
    print('\n')
    for idx, choice in enumerate(data['choices']):
        if idx == int(data['gold']):
            print(f"[[{choice}]]")
        else:
            print(choice)
    print('\n')

    print(f"Resposta do modelo:")
    print('\n')
    print(answer_model)
    #print(data['full_topic'])

In [12]:
make_question(95)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Pergunta: Qual é a reação muito comum (>1/10) que pode ocorrer com o uso de Oxaliplatina e é relacionada à função hematológica?


Alternativas:


Fosfatase alcalina anormal
Dor abdominal
Constipação
[[Anemia]]


Resposta do modelo:


<s> [INST] Given this context: ('Bula de FOLFOX4  COMPOSIÇÃO Cada frasco-ampola de 50 mg contém: oxaliplatina..................................................................................................................................................................................... 50 mg excipiente (lactose monoidratada)................................................................................................................................................ 450 mg Cada frasco-ampola de 100 mg contém: oxaliplatina................................................................................................................................................................................... 100 mg excipiente (lactose monoidratada)...............

In [13]:
make_question(109)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Pergunta: Quais são os sintomas de efeitos colaterais que podem ocorrer com o uso de Neo Mistatin creme?


Alternativas:


dores de dente ou problemas de visão
perda de peso ou dor de cabeça
falta de ar ou dor no peito
[[irritação e sensibilidade, incluindo sensação de queimação e coceira]]


Resposta do modelo:


<s> [INST] Given this context: ('Bula de Neo Mistatin  COMPOSIÇÃO Cada 4g de creme contém: nistatina.........................................................................................................................................100.000 U.I. excipientes q.s.p......................................................................................................................................... 4g (cloreto de benzalcônio, metilparabeno, propilparabeno, polawax (uniox C), borato de sódio, metabissulfito de sódio, edetato dissódico e água). 1. INDICAÇÕES Neo Mistatin creme é indicado para o tratamento de candidíase vaginal (monilíase). 2. RESULTADOS DE EFICÁCIA A nistati

In [10]:
make_question(109)

Pergunta: Qual é a reação muito comum (>1/10) que pode ocorrer com o uso de Oxaliplatina e é relacionada à função hematológica?


Alternativas:


Fosfatase alcalina anormal
Dor abdominal
Constipação
[[Anemia]]


Resposta do modelo:


<s> [INST] Given this context: ('Bula de FOLFOX4  COMPOSIÇÃO Cada frasco-ampola de 50 mg contém: oxaliplatina..................................................................................................................................................................................... 50 mg excipiente (lactose monoidratada)................................................................................................................................................ 450 mg Cada frasco-ampola de 100 mg contém: oxaliplatina................................................................................................................................................................................... 100 mg excipiente (lactose monoidratada)...............

In [15]:
make_question(213)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Pergunta: Qual é a dose máxima diária recomendada para o tratamento longo com o Cloridrato de verapamil?


Alternativas:


[[480 mg]]
600 mg
720 mg
1000 mg


Resposta do modelo:


<s> [INST] Given this context: ('Bula de Cloridrato de verapamil  COMPOSIÇÃO Cada comprimido revestido de 80 mg contém: cloridrato de verapamil.......................................................................................................................................... 80 mg excipiente* q.s.p................................................................................................................................................1 com rev *fosfato de cálcio dibásico di -hidratado, celulose microcristalina, croscarmelose sódica, dióxido de silício, estearato de magnésio, álcool polivinílico, dióxido titânio, macrogol, talco. 1. INDICAÇÕES O cloridrato de verapamil é indicado para adultos, crianças e adolescentes no tratamento de:  1. Isquemia miocárdica: a) isquemia silenciosa; b) angina crônic

In [ ]:
for jjson in list_json:
    prompt = f"Given this context: {jjson['full_topic']} this question: {jjson['query']} and this answer: {jjson['choices'][int(jjson['gold'])]} return YES if the answer is correct and NO if the answer is incorret."
    model_answer = (model.inference(prompt))
    if "YES" in model_answer:
        pass
    else:
        print(model_answer)
        print(jjson['query'])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

KeyboardInterrupt: 